In [2]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re
import time
# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [3]:
import copy
import datetime

# Loading the 2 dataset

In [ ]:
df1=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_1.csv')

In [ ]:
df2=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_2.csv')

In [ ]:
df=pd.concat([df1,df2],axis=0)

In [ ]:
df.head()

In [ ]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Links2run integration

### Loading links2run

In [ ]:
links2runs=pd.read_csv('../datasets/links2runs.csv')
del links2runs['Unnamed: 0']

In [ ]:
links2runs=links2runs.set_index(links2runs.URL)

In [ ]:
links2runs.head(3)

In [ ]:
links2runs.loc['http://services.datasport.com/2000/lauf/tdcn']

In [ ]:
links2runs.index.value_counts()

### Process link

In [ ]:
df.link=df.link.apply(lambda x: '/'.join(x.split('/')[:-1]))

In [ ]:
df.head(2)

### Groupby link

In [ ]:
links=list(df.link.value_counts().index)

In [ ]:
def add_date_name_to_group(x):
    link=x.link.iloc[0]
#     print(link)
#     print('\n\n\n\n')
    row_links2runs=links2runs.loc[link]
    try:
        row_links2runs.shape[1]
        if row_links2runs.shape[0]>1:
            row_links2runs=row_links2runs.iloc[0]
    except:
        pass
#     print(row_links2runs.shape)
    try:
        x['Date']=row_links2runs.Date
    except:
        print(row_links2runs)
        print(x)
        raise()
    x['Name']=row_links2runs.Name
    return x

In [ ]:
df.head(2)

In [ ]:
grouped_link=df.groupby(df.link)

In [ ]:
df=grouped_link.apply(add_date_name_to_group)

In [ ]:
del df['link']

In [ ]:
df.head(2)

# Delete duplicates
Some links of games are counted twice. In this case we check for duplicated rows and we remove them

In [ ]:
df=df.drop_duplicates(keep='first')

# Process temps-retards

In [ ]:
class Time:
    def __init__(self,time,mseconds=True):
        self.days,self.hours,self.minutes,self.seconds,self.mseconds=self.parse_time(time,split=True,mseconds=mseconds)
        
    def parse_time(self,time,check_only=False,split=True,mseconds=True):
        ''' Return a parsing of the time
        '''
        try :
            time.hours
            return (time.days,time.hours,time.minutes,time.seconds,time.mseconds)
        except:
            pass
        if split:
            time=time.split(' ')[0]
        if time.count(',')==0 and not check_only and mseconds:
            raise()
        time=re.split("[:.,]+",time)
        if not mseconds:
            time=time+[0]
        while len(time)<4:
            time=[0]+time
        hours,minutes,seconds,mseconds=[float(x) for x in time]
        hours+=int(minutes/60)
        minutes=minutes%60
        days=int(hours/24)
        hours=hours%24
        if not check_only:
            return (days,hours,minutes,seconds,mseconds)
        
    def get_seconds(self):  
        return self.days*3600*24+self.hours*3600 + self.minutes*60 + self.seconds + self.mseconds/1000
    
    def __str__(self):
        return str(int(self.days))+';'+str(int(self.hours))+':'+str(int(self.minutes))+'.'+str(int(self.seconds))+','+str(int(self.mseconds))
    
    def __div__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()-seconds_other
    
    def __truediv__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()/seconds_other
    

In [ ]:
dict_problems={}

In [ ]:
def apply_time(x,mseconds=True):
    try:
        time=Time(x,mseconds=mseconds)
    except:
        try:
            dict_problems[x]+=1
        except:
            dict_problems[x]=1
        return False
    return time

In [ ]:
df.temps=df.temps.apply(apply_time)

In [ ]:
df.retard=df.retard.apply(apply_time)

In [ ]:
df.pace=df.pace.apply(apply_time,args=(False,))

In [ ]:
print(Time('95.12,2'))

In [ ]:
a=Time('48.12,2')
b=Time('4.30,0')
print(a,b)

In [ ]:
b.get_seconds()

In [ ]:
a.get_seconds()

In [ ]:
a/b

In [ ]:
df['Distance']=df.temps/df.pace

In [ ]:
df.head(2)

In [ ]:
df_old=df.copy()

In [ ]:
df=df[df.Distance>0]

In [ ]:
df_old.shape

In [ ]:
df=df_old[df_old.Distance>0]

In [ ]:
df.head(3)

# Parse Date

In [ ]:
df['Year']=df.Date.apply(lambda x: x.split('.')[3])

In [ ]:
df['Month']=df.Date.apply(lambda x: x.split('.')[2])

In [ ]:
df.head(2)

# Sex Male-Female

### Get names database

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
gender_names=pd.read_csv('gender_names.csv')

In [ ]:
del gender_names['Unnamed: 0']

In [ ]:
gender_names=gender_names.set_index(gender_names.name)

In [ ]:
del gender_names['name']

In [ ]:
gender_names.head(2)

### Category from sex database

In [ ]:
category_list=list(df.cat.value_counts().index)

In [ ]:
len(category_list)

In [ ]:
category_dict_sex={}

In [ ]:
cat_to_check=['B','D','F40','F/3','H','12','10-F20']

In [ ]:
for i,cat in enumerate(category_list):
    if cat in category_dict_sex:
        continue
    names=list(df[df.cat==cat].nom.values)
    not_found=0
    male=0
    female=0
    for index,name in enumerate(names):
        splitted=re.split("[- ]+",name)
        found=0
        for name in splitted:
            try:
                row=gender_names.loc[name.lower()]
                male+=row.male_prob
                female+=row.female_prob
                found+=1
            except:
                pass
    if male!=0 or female!=0:
        male_perc=male/(male+female)
        category_dict_sex[cat]=male_perc
        if male_perc<0.85 and male_perc>0.15:
            if cat not in cat_to_check:
                cat_to_check.append(cat)
#             print('Category',cat,'Perc:',male_perc)
    if i%100==0:
#         print()
        print(i)
#         print()
    if i>7000:
        break

In [ ]:
len(cat_to_check)

In [ ]:
len(category_dict_sex)

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if category_dict_sex[cat]>0.85:
        cat_parsed+=1
        category_dict_sex[cat]='M'
    elif category_dict_sex[cat]<0.15:
        cat_parsed+=1
        category_dict_sex[cat]='F'

In [ ]:
category_dict_sex_copy=copy.deepcopy(category_dict_sex)

In [ ]:
import json

with open('category_dict_sex.json', 'w') as f:
    json.dump(category_dict_sex, f)

In [ ]:
with open('category_dict_sex.json') as f:
    category_dict_sex = json.load(f)

### Improvements

In [ ]:
category_dict_sex=copy.deepcopy(category_dict_sex_copy)

In [ ]:
to_exclude=['P-W','D','SM2/D','W','P/W']

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if cat in to_exclude:
        continue
    if category_dict_sex[cat]=='M' or category_dict_sex[cat]=='F':
        cat_parsed+=1
        continue
    cat_splitted=re.split("[-/]+",cat)
    for i in cat_splitted:
        if len(i)==3 and (i[0]=='M' or i[0]=='H') and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        elif len(i)==3 and (i[0]=='F' or i[0]=='W')  and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Fille' in i or 'Women' in i or 'Dam' in i:
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Gar' in i or 'Men' in i or 'Hom' in i: 
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        if i=='W' or i=='D' or i=='H-U20W' or i=='H-WJ':
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
cat_parsed

In [ ]:
df[df.cat=='Sch-D-M']

### Final classification - accurate

In [ ]:
del df['sex']

In [ ]:
def add_sex(x):
    cat=x.cat.iloc[0]
    try:
        sex=category_dict_sex[cat]
        if sex=='M' or sex=='F':
            x['sex']=sex
        else:
            x['sex']=False
    except:
        x['sex']=False
    return x

In [ ]:
grouped=df.groupby(df.cat)

In [ ]:
df=grouped.apply(add_sex)

In [ ]:
df.sex.value_counts()

In [ ]:
df_sex=df[df.sex!=False]

In [ ]:
df_sex.head(2)

In [ ]:
df.shape

In [ ]:
df_sex.shape

In [ ]:
min(df.Year)

In [ ]:
min(df_sex.Year)

In [ ]:
df=df.rename(columns={'Year':'RaceYear','Month':'RaceMonth'})

In [ ]:
df.head(2)

In [ ]:
df.sex.value_counts()

# Parse year (roughly eliminating clearly bad entries)

In [ ]:
df.an=df.an.apply(lambda x: False if (x=='??' or x=='????' or x=='xxxx')  else x)

In [ ]:
df.an=df.an.apply(lambda x: int(x))

In [ ]:
df=df[df.an<2017]

In [ ]:
df=df[(df.an>1870) | (df.an<100)]

# Columns names

In [ ]:
df.head(1)

In [ ]:
df.columns.values

In [ ]:
df=df[['Name','Date','RaceYear','RaceMonth','Distance','nom','sex','an','lieu','rang','temps','retard','pace']]

In [ ]:
df.head()

In [ ]:
df.columns=['Race','Date','RaceYear','RaceMonth','Distance','Name','Sex','Year','LivingPlace','Rank','Time','Delay','Pace']

In [ ]:
df=df.reset_index()

In [ ]:
del df['index']

In [ ]:
df.head(2)

In [ ]:
df.shape

# Distance

In [ ]:
df.Distance=df.Distance.apply(lambda x: float(x))

In [ ]:
df['round']=df.Distance.round()

In [ ]:
df.Distance=df.Distance.apply(lambda x: x if (abs(x-round(x)))>0.065 else round(x))

In [ ]:
df.Distance=df.Distance.apply(lambda x: 21.0975 if (abs(x-21.0975))<0.1 else x)

In [ ]:
df.Distance=df.Distance.apply(lambda x: 42.195 if (abs(x-42.195))<0.1 else x)

In [ ]:
df.Distance.value_counts()

In [ ]:
del df['round']

In [ ]:
df.head(5)

# Adding weather informations

In [ ]:
df=df[df.Sex!=False]

In [ ]:
weather_db=pd.read_csv('../datasets/races-information-weather.csv')
del weather_db['Unnamed: 0']
del weather_db['Unnamed: 0.1']

In [ ]:
weather_db.head(2)

In [ ]:
weather_db['RaceYear']=weather_db.Date.apply(lambda x: x.split('.')[3])

In [ ]:
gr=df.groupby([df.RaceYear,df.Race])

In [ ]:
def merge_weather_info(x):
    race=x.iloc[0].Race
    date=x.iloc[0].RaceYear
    weather=weather_db[(weather_db.Name==race) & (weather_db.RaceYear==date)].iloc[0]
    x['Place']=weather.Place
    x['MinTemp']=weather.min_temp
    x['MaxTemp']=weather.max_temp
    x['Weather']=weather.weather_desc
    x['RaceID']=weather.URL
    
    return x

In [ ]:
df=gr.apply(merge_weather_info)

In [ ]:
df.shape

In [ ]:
df.head(2)

# Parse times and select only meaningful features

In [ ]:
full_df=df.copy()

In [ ]:
base=datetime.datetime.strptime('0:0.0,0','%H:%M.%S,%f')

In [ ]:
full_df.Time=full_df.Time.apply(lambda x:str(x))

In [ ]:
full_df.Pace=full_df.Pace.apply(lambda x:str(x))

In [ ]:
def modify_to_parse_day(x):
    a=str(int(x[0])+1)
    return a+x[1:]

In [ ]:
full_df.Pace=full_df.Pace.apply(modify_to_parse_day)

In [ ]:
full_df.Time=full_df.Time.apply(modify_to_parse_day)

In [ ]:
full_df['Pace'] = (pd.to_datetime(full_df.Pace, format='%d;%H:%M.%S,%f')-base)

In [ ]:
max(full_df.Time)

In [ ]:
max(full_df.Distance)

In [ ]:
full_df['Time'] = (pd.to_datetime(full_df.Time, format='%d;%H:%M.%S,%f')-base)

In [ ]:
del full_df['Delay']

In [ ]:
full_df.head()

# Better distance processing

In [ ]:
df.Race.value_counts()

In [ ]:
b=full_df2[full_df2.Race=='20km de Lausanne']

In [ ]:
b.Distance.value_counts().index

In [ ]:
full_df2=full_df.copy()

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: x if x<4 or (abs(1-x/round(x)))>0.0055 else round(x))

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 21.0975 if x>4 and (abs(1-x/21.0975))<0.0055 else x)

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 42.195 if x>4 and (abs(1-x/42.195))<0.0055 else x)

In [ ]:
full_df2.groupby(full_df2.RaceYear).count()

In [ ]:
df=full_df2.copy()

# Year - trying to get better data

In [ ]:
years_values=df.Year.value_counts().index.values
years_values[years_values<120]

In [ ]:
df.Year=df.Year.apply(lambda x: 0 if x==2 else x)

In [ ]:
df[df.Year>1900].shape

In [ ]:
df[df.Year<1900].shape

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
reduced_df[reduced_df.Year==25]

In [ ]:
reduced_df.groupby(reduced_df.Year).count()

If in the name a comma is present, it is possible that the year is misunderstood.

In [ ]:
names=reduced_df.Name
list_names=[]
for name in names:
    if ',' in name:
        list_names.append(name)

In [ ]:
df['CommaInName']=df.Name.str.contains(',')

In [ ]:
df['Less120']=df.Year<120

In [ ]:
df['FalseYear']=df.CommaInName & df.Less120

In [ ]:
def false_year(x):
    if x.iloc[0].FalseYear:
        x['Year']=float('nan')
    return x

In [ ]:
df=df.groupby(df.FalseYear).apply(false_year)

In [ ]:
del df['CommaInName']
del df['Less120']
del df['FalseYear']

In [ ]:
df.head(4)

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
list(reduced_df.Year.value_counts().index)

In [ ]:
df.Year=df.Year.apply(lambda x: float('nan') if x<10 else x)

In [ ]:
min(df.Year)

In [ ]:
df.Year=df.Year.apply(lambda x: x+1900 if x<120 else x)

# Output
First output

In [ ]:
df_output=df.copy()

In [ ]:
df_output.shape

In [ ]:
df_output.head()

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split(',')[0])

In [ ]:
df_output['len_name']=df_output.Name.apply(lambda x: len(x.rstrip().lstrip().split(' ')))

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split('+')[0])

In [ ]:
df_output

In [ ]:
df_output.len_name.value_counts()

In [ ]:
df_output[df_output.len_name==6]

In [ ]:
df_output.groupby(df_output.RaceYear).count()

In [ ]:
df.groupby(df.RaceYear).count()

# Solve problem 2 runners with same name in the same race

Strategy - study duplicates:  
- Remove all fully duplicated entries (same player in different categories)
- Keep entries with different living place (different players)
- To decide what to do if same place and different times

In [15]:
df_output.Race.value_counts()

Course de l'Escalade, Genève                        153253
Schweizer Frauenlauf Bern                           118980
20km de Lausanne                                     98815
Morat-Fribourg                                       91958
Kerzerslauf                                          86338
Zürcher Silvesterlauf, Zürich                        86318
Int Greifenseelauf, Uster                            71853
Zürich Marathon, Teamrun und Cityrun                 60687
Basler Stadtlauf                                     55904
Hallwilerseelauf, Beinwil am See                     48329
Jungfrau-Marathon, Interlaken                        42855
Lausanne Marathon                                    39961
Lucerne Marathon, Luzern                             29941
ASICS Bremgarter Reusslauf                           28394
SwissCityMarathon - Lucerne, Luzern                  24460
Corrida Bulloise, Bulle                              22903
Luzerner Stadtlauf                                   226

In [4]:
df_output=pd.read_pickle('../datasets/df_year_fixed.pickle')

In [27]:
df_red=df_output.copy()#[df_output.Race=='Course de l\'Escalade, Genève']

In [28]:
df_red.shape

(1651268, 18)

In [29]:
grouped=df_red.groupby([df_red.RaceID,df_red.Name,df_output.Year])

In [30]:
duplicates=grouped.filter(lambda x: len(x)>1)

In [31]:
duplicates.shape

(4961, 18)

In [35]:
duplicates

,Race,Date,RaceYear,RaceMonth,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
25243,"Gurtenclassic, Wabern",dim. 05.11.2000,2000,11,15.099660,Siegenthaler Adrian,M,1970.0,Stettlen,37,01:13:14.100000,00:04:51,Wabern,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/gurten,2
25244,"Gurtenclassic, Wabern",dim. 05.11.2000,2000,11,10.000000,Siegenthaler Adrian,M,1970.0,Stettlen,27,00:34:48.600000,00:03:28,Wabern,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/gurten,2
34368,Kerzerslauf,sam. 17.03.2001,2001,03,15.000000,Hayoz Hugo,M,1960.0,Rotkreuz,310,01:15:08.600000,00:05:00,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,2
34369,Kerzerslauf,sam. 17.03.2001,2001,03,15.000000,Hayoz Hugo,M,1960.0,Düdingen,391,01:26:39.500000,00:05:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,2
35776,Kerzerslauf,sam. 17.03.2001,2001,03,15.000000,Röthlisberger Heinz,M,1968.0,Grosshöchstetten,60,00:55:20,00:03:41,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,2
35777,Kerzerslauf,sam. 17.03.2001,2001,03,15.000000,Röthlisberger Heinz,M,1968.0,Bätterkinden,451,01:40:12.700000,00:06:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,2
38979,"GP Fricktal, Eiken",sam. 14.04.2001,2001,04,5.853057,Bruder Daniel,M,1961.0,Basel,60,00:27:13.300000,00:04:39,Eiken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/eiken,2
38980,"GP Fricktal, Eiken",sam. 14.04.2001,2001,04,16.094492,Bruder Daniel,M,1961.0,Basel,216,01:22:21.900000,00:05:07,Eiken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/eiken,2
44437,Schweizer Frauenlauf Bern,dim. 10.06.2001,2001,06,5.000000,Bachmann Ursula,F,1954.0,Muri b. Bern,871,00:34:29.600000,00:06:53,Bern,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/frauen...,2
44438,Schweizer Frauenlauf Bern,dim. 10.06.2001,2001,06,5.000000,Bachmann Ursula,F,1954.0,Zuerich,639,00:29:53,00:05:58,Bern,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/frauen...,2


In [32]:
duplicates.Race.value_counts()

Course de l'Escalade, Genève                   964
Jungfrau-Marathon, Interlaken                  562
Zürcher Silvesterlauf, Zürich                  482
Schweizer Frauenlauf Bern                      340
Hellebardenlauf Sempach                        306
Maratona Ticino, Tenero                        274
Kerzerslauf                                    232
Gurtenclassic, Wabern                          152
Hallwilerseelauf, Beinwil am See               145
20km de Lausanne                               132
Int Greifenseelauf, Uster                      124
Zürich Marathon, Teamrun und Cityrun            90
Morat-Fribourg                                  78
BCN Tour Neuchâtel                              72
Thuner Stadtlauf, Thun                          60
Jungfrau Minirun + Minimarathon, Interlaken     52
Lucerne Marathon, Luzern                        50
ASICS Bremgarter Reusslauf                      48
Basler Stadtlauf                                46
Corrida Bulloise, Bulle        

In [38]:
duplicates[duplicates.Race=='Jungfrau-Marathon, Interlaken']

,Race,Date,RaceYear,RaceMonth,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
65404,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Bühler René,M,1959.0,Leissigen,106,04:11:24.500000,00:05:57,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2
65405,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Bühler René,M,1959.0,LI-Triesen,415,05:14:30.500000,00:07:27,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2
66872,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Müller Peter,M,1960.0,Konolfingen,497,05:40:17.400000,00:08:03,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2
66873,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Müller Peter,M,1960.0,Birsfelden,468,05:30:00.700000,00:07:49,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2
258882,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,2003,09,42.195,Zbinden Peter,M,1959.0,Spiez,484,05:43:12.700000,00:08:08,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/jungfrau,2
258883,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,2003,09,42.195,Zbinden Peter,M,1959.0,Guggisberg,27,03:58:35.400000,00:05:39,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/jungfrau,2
369229,"Jungfrau-Marathon, Interlaken",sam. 11.09.2004,2004,09,42.195,Bühler René,M,1959.0,LI-Triesen,237,04:59:54,00:07:06,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/jungfrau,2
369230,"Jungfrau-Marathon, Interlaken",sam. 11.09.2004,2004,09,42.195,Bühler René,M,1959.0,Leissigen,39,04:13:12.900000,00:06:00,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/jungfrau,2
470404,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,2005,09,42.195,Zbinden Peter,M,1959.0,Guggisberg,37,04:09:37,00:05:54,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/jungfrau,2
470405,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,2005,09,42.195,Zbinden Peter,M,1959.0,Spiez,465,05:49:06.800000,00:08:16,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/jungfrau,2


In [49]:
duplicates.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace', 'Place', 'MinTemp', 'MaxTemp',
       'Weather', 'RaceID', 'len_name', 'Rank'],
      dtype='object')

In [85]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [94]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)]

In [95]:
d2.shape

(561, 18)

In [96]:
d2[d2.Race=='Zürcher Silvesterlauf, Zürich']

,Race,Date,RaceYear,RaceMonth,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name
1918064,"Zürcher Silvesterlauf, Zürich",dim. 15.12.2013,2013,12,10.0,Fischer Pascal,M,1981.0,Wolhusen,372,00:44:29,00:04:26,Zürich,-4.0,6.0,Clear,http://services.datasport.com/2013/lauf/silvester,2
1918066,"Zürcher Silvesterlauf, Zürich",dim. 15.12.2013,2013,12,10.0,Fischer Pascal,M,1981.0,Wolhusen,1205,01:01:44.600000,00:06:10,Zürich,-4.0,6.0,Clear,http://services.datasport.com/2013/lauf/silvester,2
1922102,"Zürcher Silvesterlauf, Zürich",dim. 15.12.2013,2013,12,10.0,Meyer Dirk,M,1968.0,Zürich,261,00:43:08.200000,00:04:18,Zürich,-4.0,6.0,Clear,http://services.datasport.com/2013/lauf/silvester,2
1922104,"Zürcher Silvesterlauf, Zürich",dim. 15.12.2013,2013,12,10.0,Meyer Dirk,M,1968.0,Zürich,888,00:53:03.600000,00:05:18,Zürich,-4.0,6.0,Clear,http://services.datasport.com/2013/lauf/silvester,2
1924183,"Zürcher Silvesterlauf, Zürich",dim. 15.12.2013,2013,12,10.0,Schill Martin,M,1983.0,Zürich,939,00:53:39.900000,00:05:21,Zürich,-4.0,6.0,Clear,http://services.datasport.com/2013/lauf/silvester,2
1924184,"Zürcher Silvesterlauf, Zürich",dim. 15.12.2013,2013,12,10.0,Schill Martin,M,1983.0,Zürich,769,00:50:34.100000,00:05:03,Zürich,-4.0,6.0,Clear,http://services.datasport.com/2013/lauf/silvester,2
2129912,"Zürcher Silvesterlauf, Zürich",dim. 14.12.2014,2014,12,10.0,Arnold Philipp,M,1987.0,Cham,7,00:33:27.200000,00:03:20,Zürich,2.0,10.0,Sunny,http://services.datasport.com/2014/lauf/silvester,2
2129913,"Zürcher Silvesterlauf, Zürich",dim. 14.12.2014,2014,12,10.0,Arnold Philipp,M,1987.0,Cham,30,00:36:55.900000,00:03:41,Zürich,2.0,10.0,Sunny,http://services.datasport.com/2014/lauf/silvester,2
2130373,"Zürcher Silvesterlauf, Zürich",dim. 14.12.2014,2014,12,10.0,Benkimoun Fred,M,1970.0,Uster,1207,01:02:13.200000,00:06:13,Zürich,2.0,10.0,Sunny,http://services.datasport.com/2014/lauf/silvester,2
2130374,"Zürcher Silvesterlauf, Zürich",dim. 14.12.2014,2014,12,10.0,Benkimoun Fred,M,1970.0,Uster,619,00:49:24,00:04:56,Zürich,2.0,10.0,Sunny,http://services.datasport.com/2014/lauf/silvester,2


In [93]:
d2.Distance.value_counts()

42.195000     329
5.000000      208
10.000000     208
1.000000      150
21.097500     125
15.000000     110
11.000000      27
20.000000      24
3.000000       23
2.000000       11
8.000000       10
12.000000       8
14.000000       8
100.000000      7
4.000000        6
30.000000       3
7.250000        3
9.000000        3
5.802282        2
7.253061        2
17.502203       2
6.000000        2
7.254925        2
17.531266       2
7.265117        2
7.259278        2
35.000000       2
17.503535       2
7.258894        2
5.807291        2
             ... 
1.898211        1
4.211111        1
8.610294        1
17.524248       1
5.806249        1
4.401188        1
7.264296        1
11.214290       1
7.255833        1
7.250025        1
7.265170        1
14.511689       1
5.806122        1
17.510224       1
17.545457       1
17.539778       1
15.103590       1
0.196484        1
1.396798        1
17.520575       1
1.597253        1
8.623472        1
22.672189       1
5.206774        1
7.250023  

In [73]:
d1=duplicates[duplicates.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [74]:
d1.shape

(762, 18)

In [76]:
d1.Race.value_counts()

Jungfrau-Marathon, Interlaken    492
Maratona Ticino, Tenero          266
Zürcher Silvesterlauf, Zürich      2
Walliseller Lauf, Wallisellen      2
Name: Race, dtype: int64

In [83]:
4961-4580

381

In [80]:
duplicates.shape

(4961, 18)

In [81]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

In [82]:
dup.shape

(4580, 18)

In [61]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth','Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)

In [62]:
dup.shape

(1124, 18)

In [63]:
dup[dup.Race=='Jungfrau-Marathon, Interlaken']

,Race,Date,RaceYear,RaceMonth,Distance,Name,Sex,Year,LivingPlace,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,len_name,Rank
65404,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Bühler René,M,1959.0,Leissigen,04:11:24.500000,00:05:57,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2,106
65405,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Bühler René,M,1959.0,LI-Triesen,05:14:30.500000,00:07:27,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2,415
66872,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Müller Peter,M,1960.0,Konolfingen,05:40:17.400000,00:08:03,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2,497
66873,"Jungfrau-Marathon, Interlaken",sam. 01.09.2001,2001,09,42.195,Müller Peter,M,1960.0,Birsfelden,05:30:00.700000,00:07:49,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/jungfrau,2,468
258882,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,2003,09,42.195,Zbinden Peter,M,1959.0,Spiez,05:43:12.700000,00:08:08,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/jungfrau,2,484
258883,"Jungfrau-Marathon, Interlaken",sam. 06.09.2003,2003,09,42.195,Zbinden Peter,M,1959.0,Guggisberg,03:58:35.400000,00:05:39,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2003/lauf/jungfrau,2,27
369229,"Jungfrau-Marathon, Interlaken",sam. 11.09.2004,2004,09,42.195,Bühler René,M,1959.0,LI-Triesen,04:59:54,00:07:06,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/jungfrau,2,237
369230,"Jungfrau-Marathon, Interlaken",sam. 11.09.2004,2004,09,42.195,Bühler René,M,1959.0,Leissigen,04:13:12.900000,00:06:00,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2004/lauf/jungfrau,2,39
470404,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,2005,09,42.195,Zbinden Peter,M,1959.0,Guggisberg,04:09:37,00:05:54,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/jungfrau,2,37
470405,"Jungfrau-Marathon, Interlaken",sam. 10.09.2005,2005,09,42.195,Zbinden Peter,M,1959.0,Spiez,05:49:06.800000,00:08:16,Interlaken,NaN,NaN,NaN,http://services.datasport.com/2005/lauf/jungfrau,2,465


In [84]:
df_output=df_output.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

# Sex verification
The sex classification seems working

In [ ]:
lau=df_output[(df_output.RaceID=='http://services.datasport.com/2007/lauf/Greifenseelauf')&(df_output.Distance>6)]

In [ ]:
lau.Distance.value_counts()

In [ ]:
lau.Sex.value_counts()

In [ ]:
df_output.RaceID.value_counts()

In [ ]:
df_output.shape

# Capitalization

In [109]:
'CIAO COME STAI'.title()

'Ciao Come Stai'

In [103]:
df=df_output.copy()

In [107]:
df.dtypes

Race                    object
Date                    object
RaceYear                 int64
RaceMonth                int64
Distance               float64
Name                    object
Sex                     object
Year                   float64
LivingPlace             object
Rank                     int64
Time           timedelta64[ns]
Pace           timedelta64[ns]
Place                   object
MinTemp                float64
MaxTemp                float64
Weather                 object
RaceID                  object
dtype: object

In [105]:
df.RaceYear=df.RaceYear.apply(lambda x: int(x))

In [106]:
df.RaceMonth=df.RaceMonth.apply(lambda x: int(x))

In [108]:
df.Name=df.Name.apply(lambda x: x.title())

In [113]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

In [117]:
df=df.reset_index()
del df['index']

In [119]:
del df['level_0']

In [120]:
df

,Race,Date,RaceYear,RaceMonth,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
0,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Abgottspon Peter,M,1974.0,Zermatt,233,01:02:25,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
1,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Abplanalp Michael,M,1964.0,Bern,32,00:55:11.700000,00:03:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
2,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Abt Werner,M,1947.0,Spiez,155,01:12:42.900000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
3,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Ackermann Antoinette,F,1953.0,Alterswil,48,01:22:36.700000,00:05:30,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
4,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Ackermann Hedy,F,1946.0,Alterswil FR,42,01:23:29.300000,00:05:33,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
5,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Ackermann Meinrad,M,1956.0,La Joux,35,00:56:53.100000,00:03:47,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
6,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adamina Bruno,M,1962.0,Bern,26,00:54:41.700000,00:03:38,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
7,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adatte Michel,M,1964.0,Asuel,48,00:56:36.600000,00:03:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
8,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adesris Yeshaneh,M,1973.0,ETH-Aethiopien,23,00:49:27.300000,00:03:17,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
9,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adler Hanspeter,M,1949.0,Koblenz,33,01:02:23,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers


# Output to file

In [98]:
del df_output['len_name']

In [100]:
df_output.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place', 'MinTemp',
       'MaxTemp', 'Weather', 'RaceID'],
      dtype='object')

In [121]:
df_output=df.copy()

In [122]:
df_output

,Race,Date,RaceYear,RaceMonth,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID
0,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Abgottspon Peter,M,1974.0,Zermatt,233,01:02:25,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
1,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Abplanalp Michael,M,1964.0,Bern,32,00:55:11.700000,00:03:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
2,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Abt Werner,M,1947.0,Spiez,155,01:12:42.900000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
3,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Ackermann Antoinette,F,1953.0,Alterswil,48,01:22:36.700000,00:05:30,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
4,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Ackermann Hedy,F,1946.0,Alterswil FR,42,01:23:29.300000,00:05:33,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
5,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Ackermann Meinrad,M,1956.0,La Joux,35,00:56:53.100000,00:03:47,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
6,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adamina Bruno,M,1962.0,Bern,26,00:54:41.700000,00:03:38,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
7,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adatte Michel,M,1964.0,Asuel,48,00:56:36.600000,00:03:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
8,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adesris Yeshaneh,M,1973.0,ETH-Aethiopien,23,00:49:27.300000,00:03:17,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers
9,Kerzerslauf,sam. 18.03.2000,2000,3,15.000000,Adler Hanspeter,M,1949.0,Koblenz,33,01:02:23,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers


In [123]:
df_output.to_pickle('../datasets/df_duplicates_fixed.pickle')